Going to acquire only data about left handed pitchers, going to add more columns so that the algorithm has more to process

In [2]:
#reading in original CSV file with all of the data
import pandas as pd
initial_data = pd.read_csv('./data/combined_data.csv')

In [3]:
#looking at the columns
initial_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'index', 'pitch_type', 'game_date',
       'release_speed', 'release_pos_x', 'release_pos_z', 'player_name',
       'batter', 'pitcher', 'events', 'description', 'spin_dir',
       'spin_rate_deprecated', 'break_angle_deprecated',
       'break_length_deprecated', 'zone', 'des', 'p_throws', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'pfx_x', 'pfx_z',
       'plate_x', 'plate_z', 'outs_when_up', 'inning', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'sv_id', 'vx0', 'vy0', 'vz0',
       'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed',
       'launch_angle', 'effective_speed', 'release_spin_rate',
       'release_extension', 'game_pk', 'pitcher.1', 'fielder_2.1',
       'release_pos_y', 'estimated_ba_using_speedangle',
       'estimated_woba_using_speedangle', 'woba_value', 'woba_denom',
       'babip_value', 'iso_value', 'launch_speed_angle', 'at_bat_number',
       'pitch_number', 'pitch_n

In [22]:
#subsetting the dataframe for columns that we want to analyize
filter_data = initial_data[['pitch_name', 'release_speed', 'events', 'description', 'p_throws', 'type',\
                           'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'pitch_number', \
                           'spin_rate_deprecated', 'hit_location', 'bb_type', 'release_spin_rate', 'game_date' ,\
                           'effective_speed', 'pitch_number']]

In [24]:
#going to look at only left handed pitchers
lefty_filter = filter_data.loc[filter_data.p_throws == 'L']

In [25]:
#1.7 million rows
lefty_filter.shape

(1788537, 18)

In [26]:
#dropped irrelevant pitch names

lefty_filter = lefty_filter[(lefty_filter.pitch_name != 'Intentional Ball') 
            &(lefty_filter.pitch_name != 'Pitch Out') 
            & (lefty_filter.pitch_name != 'Fastball')
            &(lefty_filter.pitch_name != 'Unknown') ]

In [27]:
lefty_filter.shape

(1781396, 18)

In [28]:
#need to drop irrelevant descriptions that mislead to model to idetinfying an effective pitch
#need to combine swinging_strike_blocked with swinging strike

lefty_filter = lefty_filter[(lefty_filter.description != 'missed_bunt')
           & (lefty_filter.description != 'pitchout')
           & (lefty_filter.description != 'swinging_pitchout')
           & (lefty_filter.description != 'pitchout_hit_into_play')
           & (lefty_filter.description != 'pitchout_hit_into_play_score')
            & (lefty_filter.description != 'foul_pitchout')
            & (lefty_filter.description != 'unknown_strike')
            & (lefty_filter.description != 'pitchout_hit_into_play_no_out')
            ]


In [29]:
lefty_filter.shape

(1780079, 18)

In [30]:
#dropping irrelevant events
lefty_filter = lefty_filter[(lefty_filter.events != 'caught_stealing_2b')
           & (lefty_filter.events != 'pickoff_caught_stealing_2b')
            & (lefty_filter.events != 'pickoff_1b')
            & (lefty_filter.events != 'catcher_interf')
            & (lefty_filter.events != 'caught_stealing_3b')
            & (lefty_filter.events != 'pickoff_caught_stealing_3b')
            & (lefty_filter.events != 'pickoff_caught_stealing_home')
            & (lefty_filter.events != 'pickoff_3b')
            & (lefty_filter.events != 'batter_interference')
            & (lefty_filter.events != 'offensive_substitution')
            & (lefty_filter.events != 'umpire_substitution')
            & (lefty_filter.events != 'at_bat_start') 
            & (lefty_filter.events != 'run') 
            & (lefty_filter.events != 'pickoff_2b')
            & (lefty_filter.events != 'caught_stealing_home')
            ]

In [31]:
#1.7 million rows now
lefty_filter.shape

(1779102, 18)

In [53]:
#launch angle and spin rate not tracked in 2014
lefty_filter.loc[lefty_filter.game_date == '2014-08-23']

,pitch_name,release_speed,events,description,p_throws,type,hit_distance_sc,launch_speed,launch_angle,effective_speed,pitch_number,spin_rate_deprecated,hit_location,bb_type,release_spin_rate,game_date,effective_speed,pitch_number
3734004,4-Seam Fastball,87.6,single,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,1.0,NaN,7.0,line_drive,NaN,2014-08-23,NaN,1.0
3734130,Changeup,81.1,double,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,3.0,NaN,7.0,line_drive,NaN,2014-08-23,NaN,3.0
3734131,4-Seam Fastball,83.3,NaN,ball,L,B,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,2.0
3734132,Curveball,74.3,NaN,called_strike,L,S,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,1.0
3734133,2-Seam Fastball,86.3,single,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,3.0,NaN,8.0,ground_ball,NaN,2014-08-23,NaN,3.0
3734134,Changeup,81.6,NaN,foul,L,S,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,2.0
3734135,2-Seam Fastball,86.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,1.0
3734136,2-Seam Fastball,86.3,walk,ball,L,B,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,4.0
3734137,Changeup,81.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,3.0
3734138,2-Seam Fastball,86.9,NaN,ball,L,B,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2014-08-23,NaN,2.0


In [64]:
#filtering out useless pitches

lefty_filter = lefty_filter[(lefty_filter.pitch_name != 'Screwball')
           & (lefty_filter.pitch_name != 'Eephus') ]

In [65]:
lefty_filter.pitch_name.value_counts()

4-Seam Fastball    600047
Slider             246045
2-Seam Fastball    240215
Changeup           226330
Sinker             188115
Curveball          149832
Cutter              83448
Knuckle Curve       29681
Split Finger         9510
Name: pitch_name, dtype: int64

In [66]:
lefty_filter.events.value_counts()

field_out                    187432
strikeout                     94522
single                        70129
walk                          35930
double                        20889
home_run                      12141
force_out                     10883
grounded_into_double_play      9714
field_error                    4379
hit_by_pitch                   3919
sac_bunt                       3320
sac_fly                        2975
triple                         1963
double_play                    1050
fielders_choice_out             796
strikeout_double_play           258
fielders_choice                 237
intent_walk                     125
other_out                        79
sac_fly_double_play              51
triple_play                      15
sac_bunt_double_play              3
Name: events, dtype: int64

In [68]:
#gonna change the description column
strike = ['called_strike','foul' ,'foul_tip','foul_bunt' ]
swinging_strike = ['swinging_strike','swinging_strike_blocked' ]
ball = ['intent_ball', 'blocked_ball']
def ChangeDescription(value):
    if value in strike:
        value = 'strike'
        return value
    elif value in swinging_strike:
        value = 'swinging_strike'
        return value
    elif value in ball:
        value = 'ball'
        return value
    else:
        return value

In [69]:
#changed description columns
lefty_filter.description = lefty_filter.description.apply(ChangeDescription)

In [70]:
lefty_filter.description.value_counts()

ball                    642969
strike                  633166
hit_into_play           211731
swinging_strike         172246
hit_into_play_no_out     75550
hit_into_play_score      38697
hit_by_pitch              3919
Name: description, dtype: int64

In [71]:
#looking at events column
lefty_filter.events.value_counts()

field_out                    187432
strikeout                     94522
single                        70129
walk                          35930
double                        20889
home_run                      12141
force_out                     10883
grounded_into_double_play      9714
field_error                    4379
hit_by_pitch                   3919
sac_bunt                       3320
sac_fly                        2975
triple                         1963
double_play                    1050
fielders_choice_out             796
strikeout_double_play           258
fielders_choice                 237
intent_walk                     125
other_out                        79
sac_fly_double_play              51
triple_play                      15
sac_bunt_double_play              3
Name: events, dtype: int64

In [91]:
out = ['field_out', 'force_out', 'sac_bunt', 'sac_fly', 'fielders_choice_out', 'fielders_choice', 'other_out']
double_play = ['grounded_into_double_play', 'double_play', 'strikeout_double_play', 'sac_fly_double_play', 'sac_bunt_double_play']
walk = ['walk', 'intent_walk']

def changeEvent(value):
    if value in out:
        value = 'out'
        return value
    elif value in double_play:
        value = 'double_play'
        return value
    elif value in walk:
        value = 'walk'
        return value
    else:
        return value

In [92]:
lefty_filter.events = lefty_filter.events.apply(changeEvent)

In [76]:
#events look good
lefty_filter.events.value_counts()

out                205485
strikeout           94522
single              70129
walk                36055
double              20889
home_run            12141
double_play         11076
field_error          4379
hit_by_pitch         3919
triple               1963
fielders_choice       237
triple_play            15
Name: events, dtype: int64

In [77]:
lefty_filter.columns

Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'pitch_number', 'spin_rate_deprecated',
       'hit_location', 'bb_type', 'release_spin_rate', 'game_date',
       'effective_speed', 'pitch_number'],
      dtype='object')

In [82]:
#column not needed and not enough data
lefty_filter.drop(columns= ['hit_distance_sc'], inplace = True)

KeyError: "['hit_distance_sc'] not found in axis"

In [83]:
lefty_filter.drop(columns=[ 'pitch_number', 'spin_rate_deprecated', 'effective_speed'], inplace = True)

In [84]:
lefty_filter.columns


Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'launch_speed', 'launch_angle', 'hit_location', 'bb_type',
       'release_spin_rate', 'game_date'],
      dtype='object')

In [85]:
lefty_filter

,pitch_name,release_speed,events,description,p_throws,type,launch_speed,launch_angle,hit_location,bb_type,release_spin_rate,game_date
60,2-Seam Fastball,90.5,single,hit_into_play_score,L,X,NaN,NaN,9.0,line_drive,NaN,2013-10-24
61,4-Seam Fastball,88.1,NaN,ball,L,B,NaN,NaN,NaN,NaN,NaN,2013-10-24
62,4-Seam Fastball,90.4,NaN,ball,L,B,NaN,NaN,NaN,NaN,NaN,2013-10-24
63,2-Seam Fastball,90.1,NaN,strike,L,S,NaN,NaN,NaN,NaN,NaN,2013-10-24
64,2-Seam Fastball,89.4,NaN,ball,L,B,NaN,NaN,NaN,NaN,NaN,2013-10-24
65,2-Seam Fastball,89.0,out,hit_into_play_score,L,X,NaN,NaN,7.0,fly_ball,NaN,2013-10-24
66,Slider,85.9,walk,ball,L,B,NaN,NaN,NaN,NaN,NaN,2013-10-24
67,Slider,82.5,NaN,strike,L,S,NaN,NaN,NaN,NaN,NaN,2013-10-24
68,4-Seam Fastball,90.0,NaN,ball,L,B,NaN,NaN,NaN,NaN,NaN,2013-10-24
69,2-Seam Fastball,89.3,NaN,strike,L,S,NaN,NaN,NaN,NaN,NaN,2013-10-24


In [94]:
statcast_data = lefty_filter.loc[lefty_filter.game_date >= '2015']

In [88]:
statcast_data.columns

Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'launch_speed', 'launch_angle', 'hit_location', 'bb_type',
       'release_spin_rate', 'game_date'],
      dtype='object')

In [95]:
statcast_data.events.value_counts()

out             86790
strikeout       43285
single          29960
walk            15892
double           8831
home_run         5753
double_play      4815
field_error      1889
hit_by_pitch     1817
triple            887
triple_play         8
Name: events, dtype: int64

In [99]:
statcast_data.bb_type.value_counts()

ground_ball    64707
line_drive     35006
fly_ball       29405
popup           9659
Name: bb_type, dtype: int64

In [103]:
statcast_data.description.value_counts()

ball                    280429
strike                  275177
hit_into_play            89517
swinging_strike          80303
hit_into_play_no_out     32247
hit_into_play_score      17013
hit_by_pitch              1817
Name: description, dtype: int64

In [ ]:
#for launch speed, launch angle, and hit location need to make NaN values = 0
#could also create bauer units
#see if other metric
#if i combine bb_type and description that will do away with batted ball type

In [101]:
statcast_data['bauer_units'] = statcast_data['release_spin_rate']/ statcast_data['release_speed']

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
#renaming description based on hit into play
import numpy as np
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play', statcast_data['bb_type'],\
                                     statcast_data['description'])
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play_no_out', statcast_data['bb_type'],\
                                     statcast_data['description'])
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play_score', statcast_data['bb_type'],\
                                     statcast_data['description'])

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [107]:
statcast_data.description.value_counts()

ball               280429
strike             275177
swinging_strike     80303
ground_ball         64707
line_drive          35006
fly_ball            29405
popup                9659
hit_by_pitch         1817
Name: description, dtype: int64

In [108]:
statcast_data

,pitch_name,release_speed,events,description,p_throws,type,launch_speed,launch_angle,hit_location,bb_type,release_spin_rate,game_date,bauer_units
2874428,Cutter,90.4,out,line_drive,L,X,79.1,22.0,7.0,line_drive,2449.0,2015-11-01,27.090708
2874429,Slider,81.5,NaN,ball,L,B,NaN,NaN,NaN,NaN,2293.0,2015-11-01,28.134969
2874430,Curveball,77.1,NaN,ball,L,B,NaN,NaN,NaN,NaN,2543.0,2015-11-01,32.983139
2874431,Cutter,87.6,NaN,strike,L,S,NaN,NaN,NaN,NaN,2513.0,2015-11-01,28.687215
2874432,Cutter,86.2,NaN,strike,L,S,NaN,NaN,NaN,NaN,2422.0,2015-11-01,28.097448
2874433,4-Seam Fastball,91.8,NaN,ball,L,B,NaN,NaN,NaN,NaN,2115.0,2015-11-01,23.039216
2874434,Cutter,88.1,single,ground_ball,L,X,89.1,3.0,9.0,ground_ball,2465.0,2015-11-01,27.979569
2874435,Curveball,77.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,2514.0,2015-11-01,32.564767
2874436,Slider,80.0,NaN,ball,L,B,NaN,NaN,NaN,NaN,NaN,2015-11-01,NaN
2874437,4-Seam Fastball,89.5,NaN,strike,L,S,75.0,35.0,NaN,NaN,2438.0,2015-11-01,27.240223


In [109]:
#still need to do something with NaN values, 
#liking this dataset a lot more though
statcast_data.to_csv('statcast_data.csv')